# Adicionar data/hora ao nome de arquivos

- **Problema**: Muitas vezes precisamos versionar/exportar arquivos (ex.: relatórios, backups, datasets) sem sobrescrever o anterior. Uma prática simples e eficaz é inserir um timestamp (data/hora) no nome do arquivo.
- **Objetivo**: Dado um caminho de arquivo (`/caminho/arquivo.ext`), gerar um novo caminho cujo nome contenha um timestamp, como `YYYY_MM_DD_HH_MM_SS#arquivo.ext` (prefixo) ou `arquivo-YYYY_MM_DD_HH_MM_SS.ext` (sufixo).
- **Solução**: Criamos a função `timestamped_path` (e o alias compatível `st`) que:
  - permite escolher **prefixo** ou **sufixo**;
  - aceita **separador** customizável (padrão `#`);
  - permite **formato** de data/hora via `strftime`;
  - retorna `Path` por padrão ou `str` com `as_str=True`.

> Este notebook demonstra a utilidade, documentação curta e exemplos de uso para aplicar rapidamente em seus scripts.



In [1]:
from __future__ import annotations
from pathlib import Path
from datetime import datetime
from typing import Literal, Optional, Union


def timestamped_path(
    path: Union[str, Path],
    *,
    position: Literal["prefix", "suffix"] = "prefix",
    separator: str = "#",
    dt: Optional[datetime] = None,
    fmt: str = "%Y_%m_%d_%H_%M_%S",
    as_str: bool = False,
) -> Union[Path, str]:
    """
    Gera um novo caminho com timestamp no nome do arquivo.

    - path: caminho original (str ou Path)
    - position: onde colocar o timestamp ("prefix" ou "suffix")
    - separator: texto entre timestamp e nome
    - dt: datetime a usar (padrão: agora)
    - fmt: formato do timestamp (strftime)
    - as_str: se True, retorna str; caso contrário, Path

    Exemplo
    -------
    >>> timestamped_path("/tmp/relatorio.csv")
    PosixPath('/tmp/2025_09_29_10_20_15#relatorio.csv')

    >>> timestamped_path("/tmp/relatorio.csv", position="suffix", separator="-")
    PosixPath('/tmp/relatorio-2025_09_29_10_20_15.csv')
    """
    original_path = Path(path)
    timestamp = (dt or datetime.now()).strftime(fmt)

    if position == "prefix":
        new_stem = f"{timestamp}{separator}{original_path.stem}"
    else:
        new_stem = f"{original_path.stem}{separator}{timestamp}"

    new_name = f"{new_stem}{original_path.suffix}"
    new_path = original_path.with_name(new_name)

    return str(new_path) if as_str else new_path


def st(path: Union[str, Path], last: bool = True, sep: str = "#") -> str:
    """
    Compat: assinatura antiga. last=True -> timestamp como prefixo.
    Retorna str para manter compatibilidade.
    """
    return timestamped_path(
        path,
        position="prefix" if last else "suffix",
        separator=sep,
        as_str=True,
    )


__all__ = ["timestamped_path", "st"]


### Utilidade: inserir timestamp em nomes de arquivos

- **Função principal**: `timestamped_path(path, position="prefix"|"suffix", separator="#", fmt="%Y_%m_%d_%H_%M_%S")`
- **Compatibilidade**: alias `st` mantido.
- **Retorno**: `pathlib.Path` (padrão) ou `str` com `as_str=True`.

In [2]:
# Exemplos de uso
p = "/tmp/relatorio.csv"
print(timestamped_path(p))
print(timestamped_path(p, position="suffix", separator="-"))
print(timestamped_path(p, fmt="%Y%m%d", as_str=True))


/tmp/2025_09_29_10_29_07#relatorio.csv
/tmp/relatorio-2025_09_29_10_29_07.csv
/tmp/20250929#relatorio.csv
